# Prediction of Tanzanian water pumps functional status

This study will focus on identifying the functional status (functional, needs repair or non-functional) of Tanzanian water pumps. The possible explanatory variables will be location, construction year, funder, type of extraction, water quality and quantity, population using it and management organization.

I picked up this challenge from the [DrivenData](https://www.drivendata.org/) competitions list because it shows a direct and pratical application of how statistical analysis can help improve services and products quality. And as an engineer, those goals will be definitely the basis of any data science case I will have to solve. Moreover, as lots of possible explanatory variables are available, this will give me the chance to apply advance tools I learnt during the [Data Analysis and Interpretation online Specialization](https://www.coursera.org/specializations/data-analysis).

Predicting accurately the water pumps functional status will help planning maintenance earlier. That in turn will increase the availability of the water point and thus the quality of life for the people depending on those water supplies.

> This Jupyter notebook will be the basis for the final report for the [Data Analysis and Interpretation Specialization](https://www.coursera.org/specializations/data-analysis)